In [6]:
!pip install scikit-optimize

In [7]:
import argparse

parser = argparse.ArgumentParser(description='PyTorch Bayesian Optimization Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=50, metavar='N',
                    help='number of epochs to train (default: 14)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR',
                    help='learning rate (default: 0.1)')
parser.add_argument('--optim', type=str, default='sgd', choices=['sgd', 'adamw'], metavar='M',
                    help='Optimization algorithm (default: sgd)')
parser.add_argument('--criterion', type=str, default='cross_entropy', choices=['cross_entropy',], metavar='M',
                    help='Loss function (default: cross_entropy)')
parser.add_argument('--gamma', type=float, default=0.1, metavar='M',
                    help='Learning rate step gamma (default: 0.1)')
parser.add_argument('--weight-decay', type=float, default=1e-4, metavar='WD',
                    help='L2 weight decay coefficeint (default: 1e-4)')
parser.add_argument('--model', type=str, default='resnet20', choices=['resnet20', 'resnet32', 'resnet44'], metavar='M',
                    help='Resnet model (default: resnet20)')

args, _ = parser.parse_known_args()
print(args)

Namespace(batch_size=128, criterion='cross_entropy', epochs=50, gamma=0.1, lr=0.1, model='resnet20', optim='sgd', test_batch_size=1000, weight_decay=0.0001)


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])

def resnet32():
    return ResNet(BasicBlock, [5, 5, 5])

def resnet44():
    return ResNet(BasicBlock, [7, 7, 7])

In [9]:
import os
import tqdm
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms

device = torch.device("cuda:0")

transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4), 
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_eval = transforms.Compose([transforms.ToTensor(), 
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

def train(model):
    train_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./data', train=True, download=True, transform=transform_train),
                                               batch_size=args.batch_size,
                                               num_workers=os.cpu_count(),
                                               pin_memory=True,
                                               shuffle=True)

    criterion = {'cross_entropy': nn.CrossEntropyLoss(), 
                }[args.criterion]
    optimizer = {'sgd': torch.optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=args.weight_decay),
                 'adamw': torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
                }[args.optim]
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[args.epochs/2, args.epochs*3/4], gamma=args.gamma)

    model.train()
    for epoch in tqdm.trange(1, args.epochs + 1):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        scheduler.step()

def evaluate(model):
    """return test accuracy"""
    test_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./data', train=False, download=True, transform=transform_eval), 
                                               batch_size=args.batch_size,
                                               num_workers=os.cpu_count(),
                                               pin_memory=True,
                                               shuffle=False)

    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_acc = correct / len(test_loader.dataset)
    return test_acc

## Run hyperparameter search
### Search space types
- `skopt.space.Real`
- `skopt.space.Integer`
- `skopt.space.Categorical`

Use log-uniform prior if the search space is very wide: `prior='log-uniform'`

### Reference for `skopt.gp_minimize()`
- https://scikit-optimize.github.io/stable/auto_examples/bayesian-optimization.html
- https://scikit-optimize.github.io/stable/modules/generated/skopt.gp_minimize.html

### Advanced (parallelized query)
- https://scikit-optimize.github.io/stable/auto_examples/parallel-optimization.html
- https://github.com/scikit-optimize/scikit-optimize/issues/737

In [ ]:
%%time
import skopt

dimensions = [skopt.space.Real(low=1e-2, high=1e0, prior='log-uniform', name='lr'),
              skopt.space.Real(low=1e-6, high=1e-2, prior='log-uniform', name='weight_decay'),
              skopt.space.Categorical(categories=['sgd', 'adamw'], name='optim')]

@skopt.utils.use_named_args(dimensions=dimensions)
def meta_objective(**kwargs): # numeric args are passed as np.array
    args.lr = float(kwargs['lr'])
    args.weight_decay = float(kwargs['weight_decay'])
    args.optim = kwargs['optim']
    
    model = resnet20().to(device)
    train(model)
    test_acc = evaluate(model)
    return -test_acc

search_result = skopt.gp_minimize(func=meta_objective,
                            dimensions=dimensions,
                            acq_func='EI', # Expected Improvement.
                            n_calls=15,
                            n_initial_points=5,
                            initial_point_generator='random',
                            verbose=True)

Iteration No: 1 started. Evaluating function at random point.
Files already downloaded and verified



 58%|█████▊    | 29/50 [08:03<05:55, 16.92s/it]

In [ ]:
{k:v for k, v in zip(search_result.space.dimension_names, search_result.x)}

## Visualize search results

### Convergence of minima

In [ ]:
import skopt.plots
fig = skopt.plots.plot_convergence(search_result)

### Evaluation points

In [ ]:
fig = skopt.plots.plot_evaluations(result=search_result)

### Partial dependence plots (PDPs) of the estimated loss function

In [ ]:
fig = skopt.plots.plot_objective(result=search_result)